<a href="https://colab.research.google.com/github/githubmosca/CURSO_IA_ML/blob/main/Challeng3_TrabFinal.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
# Conexão com o Google Drive (opcional)
from google.colab import drive
drive.mount('/content/drive')




Mounted at /content/drive


Objetivo: Montar o Google Drive no ambiente do Google Colab para acessar arquivos armazenados nele.
Explicação: Isso permite ler e escrever arquivos diretamente no Google Drive, facilitando o armazenamento e a recuperação de datasets e modelos treinados.

In [ ]:
# Instalação das bibliotecas necessárias
!pip install transformers datasets bitsandbytes accelerate peft trl "unsloth @ git+https://github.com/unslothai/unsloth.git"


Objetivo: Instalar as bibliotecas necessárias para o processamento de linguagem natural e treinamento do modelo.
Bibliotecas Instaladas:
transformers: Biblioteca que fornece modelos pré-treinados e ferramentas para processamento de linguagem natural.
datasets: Biblioteca para carregar e pré-processar conjuntos de dados.
bitsandbytes: Permite quantizar modelos e otimizar o uso de memória.
accelerate: Facilita o treinamento em múltiplos dispositivos.
peft: Implementa técnicas de fine-tuning eficientes em termos de parâmetros, como LoRA.
trl: Fornece ferramentas para treinamento com reforço e fine-tuning supervisionado.
unsloth: Biblioteca que facilita o carregamento e treinamento de modelos grandes com suporte a quantização

In [5]:

import json
import pandas as pd
import re
import unicodedata
from datasets import load_dataset
import torch
import os

# Desativar o logging do wandb
os.environ["WANDB_DISABLED"] = "true"



  Cloning https://github.com/unslothai/unsloth.git to /tmp/pip-install-vq6ht6e1/unsloth_a563d9c4ff754f4989280243239c4818
  Running command git clone --filter=blob:none --quiet https://github.com/unslothai/unsloth.git /tmp/pip-install-vq6ht6e1/unsloth_a563d9c4ff754f4989280243239c4818
  Resolved https://github.com/unslothai/unsloth.git to commit f26d4e739ed507de7a9088da53d10fd02f58d160
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


Objetivo: Importar as bibliotecas necessárias para processamento de dados e configurar o ambiente.
Desativar o logging do wandb: Evita solicitações de API Key e possíveis interrupções no fluxo de execução.


In [ ]:
TRAIN_DATA_PATH = "/content/trn.json"
TEST_DATA_PATH = "/content/tst.json"

OUTPUT_TRAIN_DATASET = "/content/formatted_train_dataset.json"
OUTPUT_TEST_DATASET = "/content/formatted_test_dataset.json"


Objetivo: Especificar os caminhos dos arquivos de entrada (datasets de treinamento e teste) e saída (datasets formatados).
Explicação: Facilita a leitura e escrita dos arquivos nos locais corretos.

In [6]:
# Função para processar e formatar o dataset
def process_and_format_dataset(input_file_path, output_file_path, sample_size=None):
    data = []
    with open(input_file_path, 'r', encoding='utf-8') as file:
        for line in file:
            try:
                # Carrega cada linha como um objeto JSON
                item = json.loads(line)
                if 'title' in item and 'content' in item:
                    title = item['title']
                    content = item['content']
                    data.append({'title': title, 'content': content})
            except json.JSONDecodeError:
                # Ignora linhas com erro de decodificação
                pass

    # Converte a lista de dicionários em um DataFrame do pandas
    df = pd.DataFrame(data)

    # Remove registros com valores nulos em 'title' ou 'content'
    df.dropna(subset=['title', 'content'], inplace=True)

    # Remove registros onde 'title' ou 'content' são strings vazias ou contêm apenas espaços em branco
    df = df[(df['title'].str.strip() != '') & (df['content'].str.strip() != '')]

    # Remove duplicatas com base nas colunas 'title' e 'content'
    df.drop_duplicates(subset=['title', 'content'], inplace=True)

    # Reseta o índice do DataFrame
    df.reset_index(drop=True, inplace=True)

    # Função para normalizar o texto
    def normalize_text(text):
        # Remove caracteres de controle
        text = ''.join(ch for ch in text if unicodedata.category(ch)[0] != 'C')
        # Remove espaços extras
        text = re.sub(r'\s+', ' ', text).strip()
        return text

    # Aplica a função de normalização nas colunas 'title' e 'content'
    df['title'] = df['title'].apply(normalize_text)
    df['content'] = df['content'].apply(normalize_text)

    # Cria as colunas 'instruction', 'input' e 'output' para o formato necessário
    df['instruction'] = "Answer the question based on the product title."
    df['input'] = df['title'].apply(lambda x: f"What is '{x}'?")
    df['output'] = df['content']

    # Se 'sample_size' estiver definido, pega uma amostra do DataFrame
    if sample_size is not None:
        df = df.sample(n=sample_size, random_state=42).reset_index(drop=True)

    # Converte para uma lista de dicionários
    formatted_data = df[['instruction', 'input', 'output']].to_dict(orient='records')

    # Salva o resultado em um arquivo JSON
    with open(output_file_path, 'w', encoding='utf-8') as output_file:
        json.dump(formatted_data, output_file, ensure_ascii=False, indent=4)

    print(f"Dataset processado e salvo em '{output_file_path}'. Total de registros: {len(df)}")

# Processa e formata os datasets de treinamento e teste com uma amostra de registros
process_and_format_dataset(TRAIN_DATA_PATH, OUTPUT_TRAIN_DATASET, sample_size=10)  # Usando 10 registros para treinamento
process_and_format_dataset(TEST_DATA_PATH, OUTPUT_TEST_DATASET, sample_size=5)     # Usando 5 registros para teste



Objetivo: Processar os arquivos de dados, limpando, normalizando e formatando os registros para o modelo.

Passos da Função:

Leitura dos Dados: Carrega cada linha do arquivo JSON e extrai os campos 'title' e 'content'.
Criação do DataFrame: Converte a lista de dicionários em um DataFrame do pandas.
Limpeza dos Dados:
Remove registros com valores nulos ou vazios em 'title' ou 'content'.
Remove duplicatas para evitar redundância.
Normalização do Texto:
Remove caracteres de controle e espaços extras.
Mantém a capitalização e pontuação para compatibilidade com o modelo.
Criação das Colunas Necessárias:
'instruction': Instrução geral para o modelo.
'input': Pergunta formatada com base no título do produto.
'output': Descrição do produto.
Amostragem (Opcional):
Se sample_size for especificado, seleciona uma amostra dos dados para acelerar o teste.
Salvamento dos Dados Formatados: Escreve os dados processados em um arquivo JSON.


In [8]:
# Carrega os datasets formatados a partir dos arquivos JSON
train_dataset = load_dataset("json", data_files=OUTPUT_TRAIN_DATASET, split="train")
test_dataset = load_dataset("json", data_files=OUTPUT_TEST_DATASET, split="train")



Generating train split: 0 examples [00:00, ? examples/s]

Generating train split: 0 examples [00:00, ? examples/s]

Objetivo: Carregar os datasets formatados em objetos Dataset da biblioteca datasets.
Explicação: Isso permite manipular os dados de forma eficiente e aplicar transformações posteriores.

In [9]:
# Define o template de prompt
alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""



Objetivo: Definir o formato do prompt que será fornecido ao modelo durante o treinamento e inferência.
Explicação: Seguir um template consistente ajuda o modelo a entender melhor a estrutura da tarefa.

In [10]:
# Função para formatar os prompts
def formatting_prompts_func(examples):
    instructions = examples["instruction"]
    inputs = examples["input"]
    outputs = examples["output"]
    texts = []
    for instruction, input_text, output in zip(instructions, inputs, outputs):
        # Formata o texto usando o template
        text = alpaca_prompt.format(instruction, input_text, output)
        texts.append(text)
    return { "text": texts }



Objetivo: Aplicar o template de prompt a cada registro do dataset.

Passos da Função:

Itera sobre as listas de 'instruction', 'input' e 'output'.
Formata o texto usando o alpaca_prompt.
Retorna um dicionário com a chave 'text' contendo os prompts formatados.

In [11]:
# Aplica a formatação nos datasets de treinamento e teste
train_dataset = train_dataset.map(formatting_prompts_func, batched=True, remove_columns=train_dataset.column_names)
test_dataset = test_dataset.map(formatting_prompts_func, batched=True, remove_columns=test_dataset.column_names)



Map:   0%|          | 0/10 [00:00<?, ? examples/s]

Map:   0%|          | 0/5 [00:00<?, ? examples/s]

In [12]:
!pip install unsloth
# Importa as bibliotecas necessárias para o modelo
from unsloth import FastLanguageModel, is_bfloat16_supported
from transformers import TrainingArguments
from trl import SFTTrainer

Objetivo: Importar classes e funções necessárias para carregar e treinar o modelo utilizando técnicas avançadas de otimização.

In [13]:
# Configurações do modelo
max_seq_length = 512
dtype = None
load_in_4bit = True

==((====))==  Unsloth 2024.11.7: Fast Mistral patching. Transformers = 4.46.2.
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.5.1+cu121. CUDA = 7.5. CUDA Toolkit = 12.1.
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.28.post3. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/4.14G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/157 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/137k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/587k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/446 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

Explicação:

FastLanguageModel: Classe que facilita o carregamento de modelos grandes com suporte a quantização.
max_seq_length: Define o comprimento máximo das sequências de entrada.
dtype: Tipo de dado (pode ser torch.float16 ou torch.float32).
load_in_4bit: Indica que o modelo será carregado com quantização em 4 bits para economizar memória.

In [ ]:
# Carrega o modelo e o tokenizer usando o unsloth
model_name = "unsloth/mistral-7b-v0.3-bnb-4bit"  # Modelo quantizado em 4 bits
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name=model_name,
    max_seq_length=max_seq_length,
    dtype=dtype,
    load_in_4bit=load_in_4bit,
)


Objetivo: Carregar o modelo pré-treinado Mistral quantizado em 4 bits e o tokenizador correspondente.
Explicação:
Modelo Mistral 7B: Modelo de linguagem com 7 bilhões de parâmetros, adequado para geração de texto.
Quantização em 4 bits: Reduz o consumo de memória, permitindo o treinamento em GPUs com menos VRAM.

In [14]:
# Configura o modelo para Fine-Tuning com LoRA
model = FastLanguageModel.get_peft_model(
    model,
    r=16,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj",
                    "gate_proj", "up_proj", "down_proj"],
    lora_alpha=16,
    lora_dropout=0,
    bias="none",
    use_gradient_checkpointing="unsloth",
    random_state=42,
    use_rslora=False,
    loftq_config=None,
)



Unsloth 2024.11.7 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


Objetivo: Configurar o modelo para treinamento eficiente usando LoRA (Low-Rank Adaptation).
Explicação:
LoRA: Técnica que permite treinar modelos grandes atualizando apenas matrizes de baixo rank, reduzindo o número de parâmetros treináveis.
Parâmetros Importantes:
r: Rank das matrizes LoRA.
lora_alpha: Escala para as matrizes LoRA.
target_modules: Módulos do modelo que serão adaptados.
use_gradient_checkpointing: Economiza memória recalculando algumas partes durante o backpropagation.

In [15]:
# Define os argumentos de treinamento
training_args = TrainingArguments(
    output_dir="./results",                   # Diretório para salvar os resultados e checkpoints
    num_train_epochs=1,                       # Número de épocas de treinamento
    per_device_train_batch_size=1,            # Tamanho do batch de treinamento
    gradient_accumulation_steps=1,            # Acumula gradientes para simular um batch maior
    evaluation_strategy="no",                 # Desativa a avaliação durante o treinamento
    save_strategy="no",                       # Não salva checkpoints intermediários
    learning_rate=2e-4,                       # Taxa de aprendizado
    fp16=True,                                # Usa float16 (se a GPU suportar)
    logging_steps=10,                         # Log a cada 10 passos
    optim="adamw_8bit",                       # Otimizador em 8 bits para economizar memória
    report_to="none",                         # Desativa o wandb
)


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Map:   0%|          | 0/10 [00:00<?, ? examples/s]

Objetivo: Configurar os hiperparâmetros e estratégias para o treinamento do modelo.
Parâmetros Importantes:
num_train_epochs: Número de épocas de treinamento.
per_device_train_batch_size: Tamanho do lote por dispositivo (GPU).
gradient_accumulation_steps: Número de passos para acumulação de gradientes (simula um batch maior).
fp16: Utiliza precisão de 16 bits para economizar memória.
optim: Otimizador a ser utilizado; adamw_8bit otimiza em 8 bits para reduzir o uso de memória.
logging_steps: Frequência de registro de logs.
evaluation_strategy e save_strategy: Desativados para acelerar o treinamento durante os testes.

In [ ]:
# Define o Trainer usando o SFTTrainer
trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=train_dataset,
    args=training_args,
    max_seq_length=max_seq_length,
    packing=False,
    dataset_text_field="text",
)

Objetivo: Configurar o treinador para o fine-tuning supervisionado.
Explicação:
SFTTrainer: Trainer especializado para Supervised Fine-Tuning, fornecido pela biblioteca trl.
packing=False: Desativa o empacotamento de múltiplas sequências em uma única, o que pode ser útil para economizar memória.

In [16]:
# Inicia o treinamento
trainer.train()



==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 10 | Num Epochs = 1
O^O/ \_/ \    Batch size per device = 1 | Gradient Accumulation steps = 1
\        /    Total batch size = 1 | Total steps = 10
 "-____-"     Number of trainable parameters = 41,943,040


Step,Training Loss
10,1.555000


TrainOutput(global_step=10, training_loss=1.5549692153930663, metrics={'train_runtime': 19.0966, 'train_samples_per_second': 0.524, 'train_steps_per_second': 0.524, 'total_flos': 77496760197120.0, 'train_loss': 1.5549692153930663, 'epoch': 1.0})

Objetivo: Iniciar o processo de treinamento do modelo com os parâmetros e dados configurados.
Explicação:
Durante o treinamento, o modelo ajusta seus pesos para melhor prever as saídas desejadas dadas as entradas fornecidas.

In [17]:
# Prepara o modelo para inferência
FastLanguageModel.for_inference(model)



PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): MistralForCausalLM(
      (model): MistralModel(
        (embed_tokens): Embedding(32768, 4096, padding_idx=770)
        (layers): ModuleList(
          (0-31): 32 x MistralDecoderLayer(
            (self_attn): MistralAttention(
              (q_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=4096, out_features=4096, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Identity()
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=4096, out_features=16, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=16, out_features=4096, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
              (k_proj):

Objetivo: Configurar o modelo para o modo de inferência após o treinamento.
Explicação:
Ajusta o modelo para uso em geração de texto, desativando componentes específicos do treinamento.

In [18]:
# Salva o modelo treinado
model.save_pretrained("/content/drive/MyDrive/trained_model")
tokenizer.save_pretrained("/content/drive/MyDrive/trained_model")

print("Treinamento concluído e modelo salvo.")

Treinamento concluído e modelo salvo.


Objetivo: Salvar o modelo e o tokenizador treinados para uso futuro.
Explicação:
Permite recarregar o modelo sem a necessidade de treiná-lo novamente.

In [20]:
#Teste do Modelo Treinado
# install the unsloth package
!pip install unsloth
# Carrega o modelo treinado
from unsloth import FastLanguageModel
model_name = "/content/drive/MyDrive/trained_model"

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name=model_name,
    max_seq_length=max_seq_length,
    dtype=dtype,
    load_in_4bit=load_in_4bit,
)

FastLanguageModel.for_inference(model)

# Prepara um prompt de teste
prompt = alpaca_prompt.format(
    "Answer the question based on the product title.",
    "What is 'Girls Ballet Tutu Neon Pink'?",
    ""
)

# Tokeniza e gera a saída
inputs = tokenizer(prompt, return_tensors="pt").to("cuda")
outputs = model.generate(**inputs, max_new_tokens=50)
generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)

print(generated_text)


==((====))==  Unsloth 2024.11.7: Fast Mistral patching. Transformers = 4.46.2.
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.5.1+cu121. CUDA = 7.5. CUDA Toolkit = 12.1.
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.28.post3. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Unsloth: Will load /content/drive/MyDrive/trained_model as a legacy tokenizer.


Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
Answer the question based on the product title.

### Input:
What is 'Girls Ballet Tutu Neon Pink'?

### Response:
Girls Ballet Tutu Neon Pink. This is a beautiful tutu for your little ballerina. It is made of a soft, stretchy, and comfortable material. It is a perfect tutu for your little ballerina


Objetivo: Carregar o modelo treinado e realizar um teste de inferência para verificar seu funcionamento.
Explicação:
Tokenização: Converte o prompt em tensores numéricos.
Geração: O modelo gera uma resposta baseada no prompt.
Decodificação: Converte a saída numérica de volta em texto compreensível.
Resultado: Exibe a resposta gerada pelo modelo.